In [1]:
import tensorflow as tf

# GPU 설정 확인
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# 명시적으로 GPU를 사용하도록 설정
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  0


In [4]:
import os
from PIL import Image

# A 폴더와 B 폴더 경로 설정
A_folder_path = "./A_folder/"
B_folder_path = "./B_folder/"

# 폴더에서 이미지 파일 목록 불러오기
A_image_files = [f for f in os.listdir(A_folder_path) if f.endswith('.jpg') or f.endswith('.png')]
B_image_files = [f for f in os.listdir(B_folder_path) if f.endswith('.jpg') or f.endswith('.png')]

# 이미지 불러오기 (예시로 A 폴더의 첫 번째 이미지)
example_A_image = Image.open(os.path.join(A_folder_path, A_image_files[0]))

# 이미지 확인 (옵션)
example_A_image.show()


In [5]:
# 리사이징할 이미지의 크기 설정
resize_shape = (128, 128)

# A 폴더의 모든 이미지 리사이징
A_images_resized = [Image.open(os.path.join(A_folder_path, f)).resize(resize_shape) for f in A_image_files]

# B 폴더의 모든 이미지 리사이징
B_images_resized = [Image.open(os.path.join(B_folder_path, f)).resize(resize_shape) for f in B_image_files]


In [7]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

# 이미지를 numpy 배열로 변환
A_images_array = np.array([img_to_array(img) for img in A_images_resized])
B_images_array = np.array([img_to_array(img) for img in B_images_resized])

# 데이터 정규화 (0~255 -> 0~1)
A_images_array = A_images_array.astype('float32') / 255.
B_images_array = B_images_array.astype('float32') / 255.


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

# 오토인코더 모델 구성
input_img = Input(shape=(128, 128, 3))

# 인코더 부분
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# 디코더 부분
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# 오토인코더 모델 생성
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# 모델 학습 (A 폴더와 B 폴더의 이미지를 모두 사용)
autoencoder.fit(np.concatenate([A_images_array, B_images_array]), 
                np.concatenate([A_images_array, B_images_array]), 
                epochs=50, batch_size=128, shuffle=True)


Epoch 1/50
39/39 [==============================] - 146s 4s/step - loss: 0.6144
Epoch 2/50
39/39 [==============================] - 147s 4s/step - loss: 0.5541
Epoch 3/50
39/39 [==============================] - 149s 4s/step - loss: 0.5471
Epoch 4/50
28/39 [====================>.........] - ETA: 43s - loss: 0.5445

KeyboardInterrupt: 

In [ ]:
# 인코더 모델 생성
encoder = Model(inputs=autoencoder.input, outputs=encoded)

# A, B 폴더의 이미지에 대한 잠재 벡터 추출
A_latent_vectors = encoder.predict(A_images_array)
B_latent_vectors = encoder.predict(B_images_array)

# 잠재 벡터의 모양을 1차원으로 변경 (옵션)
A_latent_vectors = A_latent_vectors.reshape((A_latent_vectors.shape[0], -1))
B_latent_vectors = B_latent_vectors.reshape((B_latent_vectors.shape[0], -1))


In [ ]:
from sklearn.cluster import KMeans

# K-means 알고리즘으로 클러스터링
n_clusters = 10  # 클러스터의 개수 설정
kmeans_A = KMeans(n_clusters=n_clusters, random_state=0).fit(A_latent_vectors)
kmeans_B = KMeans(n_clusters=n_clusters, random_state=0).fit(B_latent_vectors)

# A, B 폴더의 이미지에 대한 클러스터 라벨 추출
A_cluster_labels = kmeans_A.labels_
B_cluster_labels = kmeans_B.labels_


In [ ]:
# A 폴더의 특정 상품 이미지 선택 (예: 첫 번째 이미지)
selected_A_index = 0
selected_A_cluster = A_cluster_labels[selected_A_index]

# 선택된 상품과 동일한 클러스터에 속한 B 폴더의 상품 찾기
similar_B_indices = np.where(B_cluster_labels == selected_A_cluster)[0]

# 유사한 상품의 이미지 목록 출력
for idx in similar_B_indices:
    similar_image = B_images_resized[idx]
    similar_image.show()  # 이미지 출력 (옵션)
